In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai

In [ ]:
df = pd.read_csv("Post45_NEAData_Final.csv")

In [ ]:
df.head()

In [ ]:
# how many don't have gender listed?
df[df.gender == "unknown"]

In [ ]:
# how many don't have a race or ethnicity
print("No race", len(df[df.race_ethnicity == "uniden"]))
print("Race", len(df[df.race_ethnicity != "uniden"]))

In [ ]:
df.race_ethnicity.value_counts()

In [ ]:
df.race_ethnicity = df.race_ethnicity.replace("uniden / ","", regex = True)

In [ ]:
r_e = df.race_ethnicity.value_counts()
r_e

In [ ]:
race = r_e[1:10]
race.plot.bar(title = "Race of NEA Creative Writing Recipients")

In [ ]:
ba = df.ba.value_counts()[:11]
ba

In [ ]:
print("ba listed ",len(df) - df.ba.isna().sum())
print("ba listed ", ((len(df) - df.ba.isna().sum())/len(df))*100,"%")

In [ ]:
ba.plot.bar()

In [ ]:
ivies = ["Brown University", "Columbia University", "Cornell University", "Dartmouth College", 
"Harvard University", "University of Pennsylvania", "Princeton University", "Yale University"]

df["Ivy"] = df.ba.isin(ivies)
df.Ivy.value_counts()

In [ ]:
ivies = ["Brown University", "Columbia University", "Cornell University", "Dartmouth College", 
"Harvard University", "University of Pennsylvania", "Princeton University", "Yale University"]
tier_1_schools = [
    "Stanford University",
    "Massachusetts Institute of Technology (MIT)",
    "California Institute of Technology (Caltech)",
    "University of Chicago",
    "Duke University",
    "Northwestern University",
    "University of California, Berkeley (UC Berkeley)",
    "University of California, Los Angeles (UCLA)",
    "University of Michigan, Ann Arbor",
    "University of Texas at Austin",
    "Carnegie Mellon University",
    "University of Southern California (USC)",
    "Johns Hopkins University",
    "Georgia Institute of Technology (Georgia Tech)",
    "University of North Carolina at Chapel Hill"
]

tier_2_schools = [
    "University of Washington",
    "University of Wisconsin-Madison",
    "University of Illinois Urbana-Champaign",
    "University of Florida",
    "Pennsylvania State University (Penn State)",
    "Ohio State University",
    "Purdue University",
    "University of Maryland, College Park",
    "Texas A&M University",
    "University of Minnesota, Twin Cities",
    "University of Virginia",
    "University of California, San Diego (UCSD)",
    "University of California, Davis (UC Davis)",
    "Boston University",
    "University of Rochester",
    "Michigan State University",
    "Indiana University Bloomington",
    "University of Colorado Boulder",
    "University of Arizona",
    "University of Utah"
]

top_schools = ivies + tier_1_schools

# Define a function to fill the new column based on string matching
def fill_new_column(row):
    if row['ba'] in ivies:
        return "Ivy League"
    elif row['ba'] in tier_1_schools:
        return "Tier 1"
    elif row['ba'] in tier_2_schools:
        return "Tier 2"
    else:
        return "not_found"

# Apply the function to create a new column
df['Tier'] = df.apply(fill_new_column, axis=1)

df.Tier.value_counts()

In [ ]:
top_schools = ivies + tier_1_schools

# Define a function to fill the new column based on string matching
def fill_new_column_top(row):
    if row['ba'] in top_schools:
        return "Top School"
    else:
        return "Other"
        
# Apply the function to create a new column
df['Top_schools'] = df.apply(fill_new_column_top, axis=1)

top_schools = df.Top_schools.value_counts(normalize=True)
top_schools

In [ ]:
top_schools.plot.pie()

# NYTimes

In [2]:
nyt = pd.read_csv("nyt_full.tsv", sep = '\t')
nyt.head()

,year,week,rank,title_id,title,author
0,1931,1931-10-12,1,6477,THE TEN COMMANDMENTS,Warwick Deeping
1,1931,1931-10-12,2,1808,FINCHE'S FORTUNE,Mazo de la Roche
2,1931,1931-10-12,3,5304,THE GOOD EARTH,Pearl S. Buck
3,1931,1931-10-12,4,4038,SHADOWS ON THE ROCK,Willa Cather
4,1931,1931-10-12,5,3946,SCARMOUCHE THE KING MAKER,Rafael Sabatini


In [3]:
nyt.author.nunique()

2210

In [4]:
print(nyt.author)

0                        Warwick Deeping
1                       Mazo de la Roche
2                          Pearl S. Buck
3                           Willa Cather
4                        Rafael Sabatini
                      ...               
60381                             Halsey
60382                       Brit Bennett
60383                        Delia Owens
60384                    Fredrik Backman
60385    Clive Cussler and Boyd Morrison
Name: author, Length: 60386, dtype: object


In [5]:
nyt_authors = nyt.author.unique().tolist()

# Because GPT can only do so many at once
import itertools as it

cnt = 0
auth50 = list(it.batched(nyt_authors, 50))
for item in auth50:
    cnt+=1
    c = str(cnt)
    with open("nytimes_auth/authors"+c+".txt", "w") as output:
        output.write(str(item))


# Use GPT to gather data about the authors

In [6]:
nyt_authors_short = nyt_authors[:5]
print(nyt_authors_short)

['Warwick Deeping', 'Mazo de la Roche', 'Pearl S. Buck', 'Willa Cather', 'Rafael Sabatini']


In [12]:
myapikey = open('my_gpt_apikey.txt').readline().rstrip()

client = openai.OpenAI(api_key = myapikey)

output = []
for author in nyt_authors_short:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a literary historian. You reply with only the year, not a complete sentence"},
            {
                "role": "user",
                "content": "What is the birth year of "+author
            }
        ]
    )
    my_response = completion.choices[0].message.content
    output.append(my_response)



print(output)

['1877', '1879', '1892', '1873', '1875']


In [ ]:
with open("../nytimes/authors.txt", "w") as output:
    output.write(str(nyt_authors))

In [ ]:
nyt.title.nunique()

In [ ]:
author_demo = pd.read_csv("../nytimes/authors_demographics.csv")

In [ ]:
author_demo.head()

In [ ]:
author_demo = author_demo.replace('NaN', pd.NA)

In [ ]:
author_demo.head()